In [ ]:
# This is a memo of ANN, CNN-Con1D, CNN-Con2D and ENN

In [ ]:
# Always do this step first. It authrize you the path below.
from google.colab import drive
drive.mount('/content/drive/', force_remount=True)

In [ ]:
#must have this part if using the local files and import local modules!!

rpath='/content/drive/My Drive/ML-colab/' #the root path.
fpath= rpath + 'deep-learn/image-class-CNN/'  #the path for data file

import sys  #utilized the auto-selection module that I developed
sys.path.append(rpath+'ML-tune/')  #append path for ML-tune. 
import ml_tune as tune  #A module to select the best model & hyperparameter 
import ml_utility as ut  #a module to plot, trasnform, select features... 

In [ ]:
import keras
keras.__version__

from keras.models import Sequential
from keras.layers import Conv2D, Activation, MaxPooling2D, Dropout, Dense 
from keras.layers import Conv1D, BatchNormalization, MaxPooling2D,Flatten,LSTM
from keras.constraints import maxnorm
from keras.optimizers import SGD, RMSprop, Adam
from keras.utils import np_utils, to_categorical
from keras import backend as K
#K.set_image_dim_ordering('th')
from sklearn.model_selection import GridSearchCV
from keras.wrappers.scikit_learn import KerasClassifier
#from keras.datasets import cifar10  #load data
import numpy as np
np.random.seed(1337) # for reproducibility


In [ ]:
import numpy as np
import pandas as pd
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline


In [ ]:
#----- ANN----Dense only.  (this can be compared with the traditional method)

#standarize the training and testing data (Robust, MinMax, Standard)
#NN needs numpy data type. (it seems that 'MinMax' is better)
#X_train, X_test=ut.data_scalor(X_train0, X_test0, data_type='np',scale_type='MinMax')

#--------------------------

model=Sequential()

#convolution layer
model.add(Dense(units=32, activation='relu', input_dim=X_train.shape[1]))
#model.add(Dropout(0.2)) 
model.add(Dense(units=64, activation='relu'))
#model.add(Dropout(0.3)) 
model.add(Dense(units=128, activation='relu'))
model.add(Dense(units=1, activation='sigmoid'))
print(model.summary())


#model.compile(loss='binary_crossentropy', optimizer='sgd', metrics=['accuracy'])
model.compile(loss='binary_crossentropy', optimizer=Adam(lr=0.00005), metrics=['accuracy'])

#history=model.fit(X_train, y_train, epochs=10, validation_data=(X_test, y_test), verbose=1)
#or use validation_split , batch_size=10,
history=model.fit(X_train, y_train,  epochs=50, validation_split=0.25, verbose=1)

In [ ]:
#----- CNN----Conv1D.  (this can be compared with the traditional method)

#data must be reshaped:
x_train=np.array(x_train).reshape(x_train.shape[0],x_train.shape[1],1)
x_test=np.array(x_test).reshape(x_test.shape[0],x_test.shape[1],1)

#build model !
from keras import regularizers
np.random.seed(1337)
print('Building model...')

#creat the architecture
model=Sequential()

#convolution layer
model.add(Conv1D(filters=16, kernel_size=3, activation='relu', input_shape=(x_train[0].shape)))
model.add(BatchNormalization())
model.add(Dropout(0.2))  #drop 20% neurons randomly

#the second layer
model.add(Conv1D(filters=32, kernel_size=3, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.2))  #drop 50% neurons randomly
#the second layer
model.add(Conv1D(filters=64, kernel_size=3, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.3))  #drop 50% neurons randomly

#Flatten layer
model.add(Flatten())
model.add(Dense(100, activation='relu', kernel_regularizer=regularizers.l2(0.09)))
model.add(Dropout(0.3))  #drop 50% neurons randomly
#model.add(Dense(64, activation='relu'))

#Note: here the activation makes a huge difference
#model.add(Dense(1, activation='softmax'))
model.add(Dense(1, activation='sigmoid'))

model.summary()

#-----------#compile the model
#model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
#model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.compile(loss='binary_crossentropy', optimizer=Adam(lr=0.0005), metrics=['accuracy'])

#train the model. assign it to history so as to use it later!
#history=model.fit(x_train, y_train, batch_size=10, epochs=50, validation_split=0.3)
history=model.fit(x_train, y_train, epochs=35, validation_data=(x_test, y_test),
                  verbose=1, shuffle=False)


In [ ]:
#----- CNN----Conv2D.  (this is for image classificatin)


def swish_activation(x):
    return (K.sigmoid(x) * x)

model = Sequential()
model.add(Conv2D(16, (3, 3), activation='relu', padding="same", input_shape=X_train.shape[1:]))
model.add(Conv2D(16, (3, 3), padding="same", activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(32, (3, 3), activation='relu', padding="same", input_shape=X_train.shape[1:]))
model.add(Conv2D(32, (3, 3), padding="same", activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3), activation='relu', padding="same"))
model.add(Conv2D(64, (3, 3), padding="same", activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(96, (3, 3), dilation_rate=(2, 2), activation='relu', padding="same"))
model.add(Conv2D(96, (3, 3), padding="valid", activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(128, (3, 3), dilation_rate=(2, 2), activation='relu', padding="same"))
model.add(Conv2D(128, (3, 3), padding="valid", activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())

model.add(Dense(64, activation=swish_activation, kernel_regularizer=regularizers.l2(0.01)))
model.add(Dropout(0.4))
model.add(Dense(2 , activation='sigmoid'))


print(model.summary())

model.compile(loss='binary_crossentropy', optimizer=Adam(lr=0.00005), metrics=['accuracy'])
history1=model.fit(X_train, y_train, epochs=6, batch_size = 256, 
                   validation_data=(X_test, y_test), verbose=1)


In [ ]:
##One-Hot Encoding: convert labels (uint8) to categorical numbers.
#otherwise, the numbers will be considered as integers
#looks like 10 numbers ()
y_train_oh=to_categorical(y_train)
y_test_oh=to_categorical(y_test)
print ('y_train.dtype:',y_train.dtype, " , " , 'y_train_oh.dtype:',y_train_oh.dtype )
print("before encoding:", y_train.shape, ", after Encoding:",y_train_oh.shape)


In [ ]:
#----RNN----
#RNN models (if return_sequences=True, the batch shape will be the same)
model=Sequential()
model.add(LSTM((1),batch_input_shape=(None, 5,1), return_sequences=True))
model.add(LSTM((1), return_sequences=False))
print(model.summary())

model.compile(loss='mean_absolute_error', optimizer='adam', metrics=['accuracy'])